# Flask->SQLAlchemy->DB接続テスト

In [1]:
# pip install flask

In [2]:
# pip install flask_restful

In [3]:
# pip install flask_sqlalchemy

In [4]:
import numpy as np
import pandas  as pd
import requests
import json

from flask_restful import Api, Resource
from flask import Flask, Blueprint, request ,render_template, jsonify
import flask
import json

In [5]:
from sqlalchemy import create_engine, Column, Integer, String, Time
from sqlalchemy.dialects.mysql import TIMESTAMP as Timestamp
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, scoped_session
import sqlalchemy
from flask_sqlalchemy import SQLAlchemy

## 起動確認

In [3]:
app = Flask(__name__)

@app.route("/")
def hello_world():
    return "<p>!Flask from Jupyter Notebook!</p>"

app.run(port=12345, debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:12345
Press CTRL+C to quit
127.0.0.1 - - [27/Oct/2022 06:08:59] "GET / HTTP/1.1" 200 -


## テスト用のデータベースに接続
*sqlalchemy_tutorial.ipynbを実行した後に行うこと(dbが作成されないため)

In [6]:
# データベースの設定を読み込む
from test_db_setting import engine,Base
from test_db_careate_table import User,AppLog

In [ ]:
import datetime

def strftime(p_datetime):
    """
    stringを返却
    """
    print(f"strftime {type(p_datetime)}")
    if (type(p_datetime) == "datetime.datetime"):
        return str(p_datetime.strftime("%Y-%m-%d %H:%M:%S"))
    if (type(p_datetime) == "NoneType"):
        return ""
    else:
        return str(p_datetime)

def strptime(string_time):
    """
    datetimeを返却
    """
    print(f"strptime {type(string_time)}")
    if (type(string_time) == "datetime.datetime"):
        return string_time
    elif (type(string_time) == "str"):
        print(f"common.strptime string_time={string_time}")
        return datetime.datetime.strptime(string_time, "%Y-%m-%d %H:%M:%S")
    else:
        print(f"common.strptime not string_time={string_time}")
        return datetime.datetime.strptime(string_time, "%Y-%m-%d %H:%M:%S")

## GET

## POST

## FlaskからのUserデータの参照

### <span style="color:orange">(1)getAccountの関数</span>
↓呼び出し
### <span style="color:orange">(2)ConvertToJsonの関数</span>
↓呼び出し
### <span style="color:orange">(3)getDataの関数の概要</span>
SQLAlchemyでデータベースに接続し、  
user_idでuserテーブルを検索をし、該当したUserオブジェクト群を取得する  

- Parameters
  - user_id : 検索対象のアカウントid(int)
  - operation_user_id : Webアプリケーション操作ユーザーのアカウントid(int)
- Returns
  - userオブジェクトのリスト

↓return
### <span style="color:orange">(4)ConvertToJsonの関数の概要</span>
getDataの関数のreturnをjson形式に変換

- Parameters
  - user_id : 検索するアカウントのアカウントID(int)
  - operation_user_id : Webアプリケーション操作アカウントのID(int)
- Returns
  - result_json_dict(json形式のdict型取得したアカウント情報)
```
{
  "body": {
    "name": "user",
    "id": <User.id>,
    "user_name": <User.name>,
    "created_at": <User.created_at>,
    "updated_at": <User.updated_at>
},
  "status": {
    "code" : "I0001",
    "message" : "",
    "detail" : ""
  }
}
```
↓return
### <span style="color:orange">(5)getAccountの関数の概要</span>
ConvertToJsonの関数のreturnのdict型のデータをjsonifyでよしなにjsonに変換する

<br>

*json.dumpとの違い  
headerのcontent-typeを指定しなくていい。  
→flask使うならめんどくさいことしなくていい  

In [15]:
def getData(user_id, operation_user_id):
    Session = sessionmaker(bind=engine)
    ses = Session()
    
    res = ses.query(User).get(user_id)
    ses.close()
    return res

In [18]:
def ConvertToJson(user_id, operation_user_id):
    result = getData(user_id, operation_user_id)
    # TODO モデルの検索結果(正常・異常)によってレスポンスの出力内容を変える
    result_json_dict = {
        "body": {
            "name": "user",
            "id": user_id,
            "user_name": result.name,
            "created_at": result.created_at.strftime("%Y-%m-%d %H:%M:%S"),
            "updated_at": result.updated_at.strftime("%Y-%m-%d %H:%M:%S")
        },
        "status": {
            "code" : "I0001",
            "message" : "successful!",
            "detail" : ""
        }
    }
    return result_json_dict


In [21]:
system_account_id=999

app = Flask(__name__)
@app.route('/account/get/<id>', methods=['GET'])
def getAccount(id):
    account_json = ConvertToJson(id, system_account_id)
    return jsonify(account_json)

In [22]:
app.run(port=12345, debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:12345
Press CTRL+C to quit
127.0.0.1 - - [27/Oct/2022 10:25:56] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [27/Oct/2022 10:26:01] "GET / HTTP/1.1" 404 -


2022-10-27 10:26:30,398 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-27 10:26:30,403 INFO sqlalchemy.engine.Engine SELECT users.created AS users_created, users.modified AS users_modified, users.id AS users_id, users.name AS users_name, users.age AS users_age, users.email AS users_email 
FROM users 
WHERE users.id = ?
2022-10-27 10:26:30,405 INFO sqlalchemy.engine.Engine [cached since 312.4s ago] ('4',)
2022-10-27 10:26:30,428 INFO sqlalchemy.engine.Engine ROLLBACK


127.0.0.1 - - [27/Oct/2022 10:26:30] "GET /account/get/4 HTTP/1.1" 200 -


2022-10-27 10:26:35,152 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-27 10:26:35,155 INFO sqlalchemy.engine.Engine SELECT users.created AS users_created, users.modified AS users_modified, users.id AS users_id, users.name AS users_name, users.age AS users_age, users.email AS users_email 
FROM users 
WHERE users.id = ?
2022-10-27 10:26:35,157 INFO sqlalchemy.engine.Engine [cached since 317.2s ago] ('2',)
2022-10-27 10:26:35,181 INFO sqlalchemy.engine.Engine ROLLBACK


...

127.0.0.1 - - [27/Oct/2022 10:26:45] "GET /account/get/2 HTTP/1.1" 200 -


2022-10-27 10:26:49,494 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-27 10:26:49,499 INFO sqlalchemy.engine.Engine SELECT users.created AS users_created, users.modified AS users_modified, users.id AS users_id, users.name AS users_name, users.age AS users_age, users.email AS users_email 
FROM users 
WHERE users.id = ?
2022-10-27 10:26:49,500 INFO sqlalchemy.engine.Engine [cached since 331.5s ago] ('1',)
2022-10-27 10:26:49,517 INFO sqlalchemy.engine.Engine ROLLBACK


[2022-10-27 10:26:49,529] ERROR in app: Exception on /account/get/1 [GET]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.10/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.10/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.10/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/tmp/ipykernel_14895/3705283127.py", line 9, in getAccount
    account_json = ConvertToJson(id, system_account_id)
  File "/tmp/ipykernel_14895/3801739938.py", line 38, in ConvertToJson
    "user_name": result.name,
AttributeError: 'NoneType' object has no attribute 'name'
127.0.0.1 - - [27/Oct/2022 10:26:49] "

2022-10-27 10:26:55,458 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-27 10:26:55,460 INFO sqlalchemy.engine.Engine SELECT users.created AS users_created, users.modified AS users_modified, users.id AS users_id, users.name AS users_name, users.age AS users_age, users.email AS users_email 
FROM users 
WHERE users.id = ?
2022-10-27 10:26:55,469 INFO sqlalchemy.engine.Engine [cached since 337.5s ago] ('13',)
2022-10-27 10:26:55,480 INFO sqlalchemy.engine.Engine ROLLBACK


127.0.0.1 - - [27/Oct/2022 10:26:55] "GET /account/get/13 HTTP/1.1" 200 -


## Flaskからのデータのinsert

### <span style="color:orange">(1)createAccountの関数の概要</span>
flaskのrequestで.jsonして読み込みし、payload変数に格納

↓呼び出し
### <span style="color:orange">(2)createJsonの関数の概要</span>
jsonをdictに格納

↓呼び出し
### <span style="color:orange">(3)insertDataの関数の概要</span>
dictをSQLAlchemyでデータベースにaddする.  
できたらTrue.  
できなかったら保留状態を破棄して、データベースへの反映は全くせず,Falseを返す

↓return
### <span style="color:orange">(4)createJsonの関数の概要</span>
True,Falseでstatusのcodeを格納し、dict型でreturn

↓return
### <span style="color:orange">(5)createAccountの関数の概要</span>
dict型のデータをjsonifyでよしなにjsonに変換

In [29]:
import datetime

In [30]:
def insertData(user_dict, operation_user_id):
    user = User()
    user.id = user_dict['user_id']
    user.name = user_dict['user_name']
    user.created_at = user_dict['created_at']
    user.updated_at = user_dict['updated_at']
    # user.status = user_dict['status']
    Session = sessionmaker(bind=engine)
    ses = Session()
    ses.begin()
    try:
        ses.add(user)
        ses.commit()
        res = True
    except:
        ses.rollback()
        res = False
    finally:
        ses.close()
    return res

In [31]:
def createJson(user_request, operation_user_id):
    user = {
        'user_id' : user_request['user_id'],
        'user_name' : str(user_request['user_name']),
        'created_at' : datetime.datetime.now(),
        'updated_at' : datetime.datetime.now(),
        # 'status' :  Status.getStatusKey("NEW")
    }

    try:
        if insertData(user, operation_user_id) == True:
            code="I0001"
            message="Created user Succesfuly."
        else:
            code="E0001"
            message=""
    except:
        code="E0009"
        message="Created failed"

    result_json = {
        "body": "",
        "status": {
            "code" : code,
            "message" : message,
            "detail" : ""
        }
    }
    return result_json

In [32]:
@app.route('/account/create', methods=['POST'])
def createAccount():
    #payload = request.data.decode('utf-8')
    payload = request.json
    print(f"payload={payload}")
    response_json = createJson(payload, system_account_id)
    return jsonify(response_json)


AssertionError: View function mapping is overwriting an existing endpoint function: createAccount

In [37]:
app.run(port=12446, debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:12446
Press CTRL+C to quit


payload={'user_id': 11111, 'user_name': 'flask_create_user_test'}
2022-10-27 14:38:48,427 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-27 14:38:48,433 INFO sqlalchemy.engine.Engine INSERT INTO users (id, name, age, email, created, modified) VALUES (?, ?, ?, ?, ?, ?)
2022-10-27 14:38:48,434 INFO sqlalchemy.engine.Engine [generated in 0.00143s] (11111, 'flask_create_user_test', None, None, '2022-10-27 14:38:48.422607', '2022-10-27 14:38:48.422612')
2022-10-27 14:38:48,449 INFO sqlalchemy.engine.Engine COMMIT


127.0.0.1 - - [27/Oct/2022 14:38:48] "POST /account/create HTTP/1.1" 200 -


```
curl http://127.0.0.1:12446/account/create -X POST -H "Content-Type: application/json" --data '{"user_id": 11111,"user_name": "flask_create_user_test"}'
```

In [38]:
ConvertToJson(11111,12345)

2022-10-27 14:40:05,011 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-27 14:40:05,013 INFO sqlalchemy.engine.Engine SELECT users.created AS users_created, users.modified AS users_modified, users.id AS users_id, users.name AS users_name, users.age AS users_age, users.email AS users_email 
FROM users 
WHERE users.id = ?
2022-10-27 14:40:05,016 INFO sqlalchemy.engine.Engine [cached since 1.144e+04s ago] (11111,)
2022-10-27 14:40:05,033 INFO sqlalchemy.engine.Engine ROLLBACK


{'body': {'name': 'user',
  'id': 11111,
  'user_name': 'flask_create_user_test',
  'created_at': '2022-10-27 14:38:48',
  'updated_at': '2022-10-27 14:38:48'},
 'status': {'code': 'I0001', 'message': 'successful!', 'detail': ''}}

## Flaskからのデータのupdate

### <span style="color:orange">(1)updateAccountの関数の概要</span>
flaskのrequestで.jsonして読み込みし、payload変数に格納

↓呼び出し
### <span style="color:orange">(2)updateJsonの関数の概要</span>
jsonをdictに格納

- Parameters
  - account_request : 作成するアカウント詳細(json
  - operation_account_id : Webアプリケーション操作アカウントのID(int)
- Returns
  - JSON形式の処理結果
    - 正常
    - 異常

↓呼び出し
### <span style="color:orange">(3)updateDataの関数の概要</span>
idでデータを参照し、その値をupdateされたdictの値に変更する。  
その際、.with_for_update()で排他編集にする。  
成功すれば、True。失敗すれば、rollbackしFalseを返す

↓return
### <span style="color:orange">(4)updateJsonの関数の概要</span>
True,Falseでstatusのcodeを格納し、dict型でreturn

↓return
### <span style="color:orange">(5)updateAccountの関数の概要</span>
dict型のデータをjsonifyでよしなにjsonに変換


In [ ]:
def updateData(user_dict, operation_user_id):
    user_id = user_dict.get('id')
    Session = scoped_session(sessionmaker(bind=engine, autocommit=False))
    res=False
    ses = Session()
    # with_for_update()関数でロックをかける 排他編集
    user_record = ses.query(User).with_for_update().get(user_id)
    print(f"user#update user_record={user_record}")
    message = ""
    try:
        v = user_dict.get('user_name')
        if (v != None):
            user_record.user_name=v
        
        user_record.updated_at=strftime(datetime.datetime.now())
        # v = user_dict.get('status')
        # if (v != None):
        #     user_record.status=v
        ses.add(user_record)
        #他のプロセスによるロックを待つ
        #time.sleep(1)
        ses.commit()
        res = True
    except Exception as e:
        message = str(e)
        print(f"user#update error:{message}")
        ses.rollback()
        res = False
    finally:
        ses.close()
    return (res, message)   

In [1]:
def convertdict(from_dict):
    print(f"convertdict from_dict={from_dict}")
    target_dict = {}
    if ('id' in from_dict):
        target_dict['id'] = int(from_dict['id'])
    if ('user_name' in from_dict):
        target_dict['user_name'] = str(from_dict['user_name'])
    if ('created_at' in from_dict):
        target_dict['created_at'] = strptime(from_dict['created_at'])
    if ('updated_at' in from_dict):
        target_dict['updated_at'] = strptime(from_dict['updated_at'])
    if ('status' in from_dict):
        target_dict['status'] = int(from_dict['status'])
    return target_dict

In [2]:
def updateJson(account_request, operation_account_id):
    account = convertdict(account_request)
    try:
        res = updateData(account, operation_account_id)
        print(f"AccountApi#update res={res[0]},{res[1]}")
        if res[0] == True:
            code="I0001"
            message="Updated Account Succesfuly."
        else:
            code="E0001"
            message=res[1]
        
    except Exception as e:
        code="E0009"
        message=f"Updated failed {e}"
    
    result_json = {
        "body": "",
        "status": {
            "code" : code,
            "message" : message,
            "detail" : ""
        }
    }
    return result_json

In [3]:
@app.route('/account/update', methods=['POST'])
def updateAccount():
    #payload = request.data.decode('utf-8')
    payload = request.json
    print(f"payload={payload}")
    response_json = updateJson(payload, system_account_id)
    return jsonify(response_json)


NameError: name 'app' is not defined

# ↓以下修正していない


## Flaskからのデータの検索

### <span style="color:orange">(1)updateAccountの関数の概要</span>
flaskのrequestで.jsonして読み込みし、payload変数に格納

↓呼び出し
### <span style="color:orange">(2)updateJsonの関数の概要</span>
jsonをdictに格納

- Parameters
  - account_request : 作成するアカウント詳細(json
  - operation_account_id : Webアプリケーション操作アカウントのID(int)
- Returns
  - JSON形式の処理結果
    - 正常
    - 異常

↓呼び出し
### <span style="color:orange">(3)updateDataの関数の概要</span>
idでデータを参照し、その値をupdateされたdictの値に変更する。  
その際、.with_for_update()で排他編集にする。  
成功すれば、True。失敗すれば、rollbackしFalseを返す

↓return
### <span style="color:orange">(4)updateJsonの関数の概要</span>
True,Falseでstatusのcodeを格納し、dict型でreturn

↓return
### <span style="color:orange">(5)updateAccountの関数の概要</span>
dict型のデータをjsonifyでよしなにjsonに変換


In [ ]:
def searchData(account_dict, operation_account_id):
    """
    dictアカウントからaccountテーブルを検索し、該当したAccountオブジェクト群を取得する

    Parameters
    ----------
    {
        'account_name' : 文字列str(self.account_name),
        'start_on' : 文字列 '2020-05-01 00:00:00',
        'end_on' : 文字列 '2020-12-31 00:00:00',
        'created_by' : account_id,
        'created_at' : 文字列 '2020-12-31 00:00:00',
        'updated_by' : account_id,
        'updated_at' : 文字列 '2020-12-31 00:00:00',
        'status' : statusの数値
    }

    Returns
    -------
    Accountオブジェクトのリスト
    """
    print(f"account_dict={account_dict}")
    Session = sessionmaker(bind=engine)
    ses = Session()
    res = None
    rs = ses.query(Account)
    v = account_dict.get('account_name')
    if (v != None):
        rs = rs.filter(Account.account_name==v)
    v = account_dict.get('start_on')
    if (v != None):
        rs = rs.filter(Account.start_on==v)
    v = account_dict.get('end_on')
    if (v != None):
        rs = rs.filter(Account.end_on==v)
    v = account_dict.get('created_by')
    if (v != None):
        rs = rs.filter(Account.created_by==v)
    v = account_dict.get('created_at')
    if (v != None):
        rs = rs.filter(Account.created_at==v)
    v = account_dict.get('updated_by')
    if (v != None):
        rs = rs.filter(Account.updated_by==v)
    v = account_dict.get('updated_at')
    if (v != None):
        rs = rs.filter(Account.updated_at==v)
    v = account_dict.get('status')
    if (v != None):
        rs = rs.filter(Account.status==v)
    rs = rs.filter(Account.status!=Status.getStatusKey("DELETE"))

    res = rs.all()
    lambda r: print(f"r={r}"),res
    ses.close()
    return res


In [ ]:

def search(request, user_id):
    """
    /account/searchで呼び出されたAPIの検索処理

    Parameters
    ----------
    account_request : json
        アカウント検索項目
    user_id : int
        Webアプリケーション操作アカウントのID

    Returns
    -------
    JSON形式の処理結果
        正常
        異常
    """

    account_request = convertdict(request)
    try:
        results = Account.search(account_request, user_id)
        code="I0001"
        message=f"Found ({len(results)}) records."
        
    except Exception as e:
        code="E0009"
        message="Search failed: " + str(e)

    result_json = {
        "body": list(map(lambda s: s.toJson(), results)),
        "status": {
            "code" : code,
            "message" : message,
            "detail" : ""
        }
    }
    return result_json

In [ ]:

@api_bp.route('/account/search', methods=['POST'])
def searchAccount():
    #payload = request.data.decode('utf-8')
    payload = request.json
    print(f"payload={payload}")
    response_json = AccountApi.search(payload, system_account_id)
    return jsonify(response_json)


## Flaskからのデータの削除

### <span style="color:orange">(1)updateAccountの関数の概要</span>
flaskのrequestで.jsonして読み込みし、payload変数に格納

↓呼び出し
### <span style="color:orange">(2)updateJsonの関数の概要</span>
jsonをdictに格納

- Parameters
  - account_request : 作成するアカウント詳細(json
  - operation_account_id : Webアプリケーション操作アカウントのID(int)
- Returns
  - JSON形式の処理結果
    - 正常
    - 異常

↓呼び出し
### <span style="color:orange">(3)updateDataの関数の概要</span>
idでデータを参照し、その値をupdateされたdictの値に変更する。  
その際、.with_for_update()で排他編集にする。  
成功すれば、True。失敗すれば、rollbackしFalseを返す

↓return
### <span style="color:orange">(4)updateJsonの関数の概要</span>
True,Falseでstatusのcodeを格納し、dict型でreturn

↓return
### <span style="color:orange">(5)updateAccountの関数の概要</span>
dict型のデータをjsonifyでよしなにjsonに変換


In [ ]:
def delete(account_id, operation_account_id):
    Session = scoped_session(sessionmaker(bind=engine, autocommit=False))
    ses = Session()
    account_record = ses.query(Account).with_for_update().get(account_id)
    try:
        account_record.status=Status.getStatusKey("DELETE")
        ses.add(account_record)
        #他のプロセスによるロックを待つ
        #time.sleep(1)
        ses.commit()
        message = ""
        res = True
    except Exception as e:
        message = str(e)
        print(f"Account#update error:{message}")
        ses.rollback()
        res = False
    finally:
        ses.close()
    return (res, message)   

In [ ]:

def delete(account_id, operation_account_id):
    """
    /account/deleteで呼び出されたAPIの検索処理

    Parameters
    ----------
    account_id : int
        削除するアカウントID
    operation_account_id : int
        Webアプリケーション操作アカウントのID

    Returns
    -------
    JSON形式の処理結果
        正常
        異常
    """

    try:
        res = Account.delete(account_id, operation_account_id)
        if res[0] == True:
            code="I0001"
            message="deleted Account Succesfuly."
        else:
            code="E0001"
            message=res[1]
        
    except Exception as e:
        code="E0009"
        message=f"Deleted failed:{str(e)}"
    
    result_json = {
        "body": "",
        "status": {
            "code" : code,
            "message" : message,
            "detail" : ""
        }
    }
    return result_json

In [ ]:

@api_bp.route('/account/delete/<id>', methods=['GET'])
def deleteAccount(id):
    account_json = AccountApi.delete(id, system_account_id)
    return jsonify(account_json)


In [ ]:
# res = ses.query(User).filter(User.id == user_id)